## INTRO

nro id : se não tiver length 16 adicionar 0 na frente

instituição
- hospital : não é de ensino, tem q marcar q não é de academia
- mesma instituição, registros e tempos diferentes  =>  tem q agrupar
similaridade de string -> função cosseno de PLN (ngramas)



1) pegar o nome de todas as instituições do brasil (usar a lista do MEC)


- fazer a tabela de frequência /
eliminar oq tem pouca frequência
- se for secretaria municipal algo assim e aparecer poucas vezes => só rotular como servidor público
- se estiver so EACH (sem ter USP no nome), categorizar como 'outros'
- lista de universidades
    - tabela do ROR
    - tabela do MEC

In [1]:
from google.colab import drive
import os

import pandas as pd
import numpy as np

drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/USP/2024_2º/MQA/')

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

Mounted at /content/gdrive


## READ_CSV

In [6]:
df = pd.read_csv('atuacoes_2.csv', encoding='UTF-8', nrows=5000) #, dtype=str, nrows=1000)

df['ANO-FIM'] = df['ANO-FIM'].fillna(2024)
df['FLAG-DEDICACAO-EXCLUSIVA'] = df['FLAG-DEDICACAO-EXCLUSIVA'].replace('NAO', False).replace('SIM', True)

df = df.astype({
    'NRO-ID-CNPQ': int,
    'CODIGO-INSTITUICAO': str,
    'NOME-INSTITUICAO': str,
    'ANO-INICIO': int,
    'ANO-FIM': int,
    'ENQUADRAMENTO-FUNCIONAL': str,
    'TIPO-DE-VINCULO': str,
    'OUTRAS-INFORMACOES': str,
    'CARGA-HORARIA-SEMANAL': float,
    'FLAG-DEDICACAO-EXCLUSIVA': bool
})

df = df.fillna(' ')
df2 = df.copy()

#df

In [3]:
# LISTA DE UNIVERSIDADES NACIONAIS
# https://dadosabertos.mec.gov.br/indicadores-sobre-ensino-superior/item/181-instituicoes-de-educacao-superior-do-brasil
# https://dadosabertos.mec.gov.br/images/conteudo/Ind-ensino-superior/2022/PDA_Lista_Instituicoes_Ensino_Superior_do_Brasil_EMEC.csv

df_uni = pd.read_csv('sup.csv', encoding='UTF-8')
df_uni_filtered = df_uni[['NOME_DA_IES', 'SIGLA']]
df_uni_filtered = df_uni_filtered.fillna('')

## TRATAMENTO

In [4]:
def clean_names(df, column_name):

    # Remove '&'
    df[column_name] = df[column_name].replace('&AMP;', ' ', regex=True)

    # Remove any non-alphanumeric characters except whitespace
    df[column_name] = df[column_name].str.replace('[^\w\s]', ' ', regex=True)

    # Normalize string
    df[column_name] = df[column_name].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

    # Remove stopwords
    stop = ['DO', 'DA', 'DE', 'E']
    df[column_name] = df[column_name].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

    return df

In [7]:
# List of lattes CV

# Clean NOME-INSTITUICAO
df2 = df2.apply(lambda x: x.astype(str).str.upper())
df2 = clean_names(df2, 'NOME-INSTITUICAO')

# Drop rows with frequency less than 50
df2 = df2[df2.groupby('NOME-INSTITUICAO')['NOME-INSTITUICAO'].transform('count').ge(50)]
df2['NOME-INSTITUICAO'].value_counts()

# Split df: education & others
df2_edu = df2.loc[df2['ENQUADRAMENTO-FUNCIONAL'].isin(['LIVRE', 'COLABORADOR', 'PROFESSOR_VISITANTE'])]
df2_outros = df2.loc[df2['ENQUADRAMENTO-FUNCIONAL'].isin(['SERVIDOR_PUBLICO', 'CELETISTA', 'NAN', 'OUTRO'])]

df2_edu

,NRO-ID-CNPQ,CODIGO-INSTITUICAO,NOME-INSTITUICAO,ANO-INICIO,ANO-FIM,ENQUADRAMENTO-FUNCIONAL,TIPO-DE-VINCULO,OUTRAS-INFORMACOES,CARGA-HORARIA-SEMANAL,FLAG-DEDICACAO-EXCLUSIVA
653,3356091188000026,006700000002,UNIVERSIDADE SAO PAULO,1999,2000,COLABORADOR,LIVRE,MEMBRO DO NÚCLEO DE PLANEJAMENTO ECONÔMICO DA,0.0,FALSE
708,3326759383856112,006700000002,UNIVERSIDADE SAO PAULO,1989,1990,COLABORADOR,LIVRE,ESTAGIÁGIO DIDÁTICO,,TRUE
710,3326759383856112,006700000002,UNIVERSIDADE SAO PAULO,2009,2018,LIVRE,LIVRE,PROFESSOR CONVIDADO,,FALSE
713,3326759383856112,006700000002,UNIVERSIDADE SAO PAULO,1978,1978,COLABORADOR,LIVRE,ESTAGIÁRIO DIDÁTICO,,TRUE
716,3326759383856112,006700000002,UNIVERSIDADE SAO PAULO,2021,2024,COLABORADOR,LIVRE,PROFESSOR SÊNIOR,,FALSE
...,...,...,...,...,...,...,...,...,...,...
4074,3340189813794279,006700000002,UNIVERSIDADE SAO PAULO,2009,2009,LIVRE,LIVRE,MONITORA,6.0,FALSE
4075,3340189813794279,006700000002,UNIVERSIDADE SAO PAULO,2008,2011,LIVRE,LIVRE,PESQUISADORA MESTRANDA,40.0,TRUE
4357,3301334284145414,006700000002,UNIVERSIDADE SAO PAULO,2010,2011,LIVRE,LIVRE,TUTOR PRESENCIAL,10.0,FALSE
4761,3386990686337829,006700000002,UNIVERSIDADE SAO PAULO,2015,2015,LIVRE,LIVRE,ASSISTÊNCIA AO PROFESSOR E ALUNOS / MONITORIA,6.0,FALSE


In [9]:
# List of Universities

df_uni_filtered = df_uni_filtered.apply(lambda x: x.astype(str).str.upper())
df_uni_filtered = clean_names(df_uni_filtered, 'NOME_DA_IES')

# merge NOME and SIGLA column
df_uni_filtered['NOME_E_SIGLA'] = df_uni_filtered['SIGLA'] + ' ' + df_uni_filtered['NOME_DA_IES']
df_uni_filtered

,NOME_DA_IES,SIGLA,NOME_E_SIGLA
0,FACULDADE TECNOLOGIA SENAC CURITIBA,,FACULDADE TECNOLOGIA SENAC CURITIBA
1,FACULDADE REGIONAL SERRANA,FUNPAC,FUNPAC FACULDADE REGIONAL SERRANA
2,UNIVERSIDADE ESTADO AMAPA,UEAP,UEAP UNIVERSIDADE ESTADO AMAPA
3,FACULDADE SAO JUDAS SAO BERNARDO CAMPO,,FACULDADE SAO JUDAS SAO BERNARDO CAMPO
4,FACULDADE CIENCIAS GERENCIAIS ALVES FORTES JUI...,FACE ALFOR JF,FACE ALFOR JF FACULDADE CIENCIAS GERENCIAIS AL...
...,...,...,...
4323,FACULDADE NEGOCIOS,FTEC NEGÓCIOS,FTEC NEGÓCIOS FACULDADE NEGOCIOS
4324,FACULDADE RONDONIA,FARO,FARO FACULDADE RONDONIA
4325,FACULDADE SAUDE TECSOMA,FASTEC,FASTEC FACULDADE SAUDE TECSOMA
4326,FACULDADE CIENCIAS GERENCIAIS VILHENA,FCGV,FCGV FACULDADE CIENCIAS GERENCIAIS VILHENA


## NLP

### Cosine

In [ ]:
# SIMILARIDADE POR SIGLA + NOME COMPLETO

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

threshold = 0.8

# Vetorização TF-IDF
vectorizer = TfidfVectorizer().fit(df_uni_filtered['NOME_E_SIGLA'])
tfidf_matrix_universidades = vectorizer.transform(df_uni_filtered['NOME_E_SIGLA'])


for i in df2_edu['NOME-INSTITUICAO']:

    similaridade_list = []

    vector_comparacao = vectorizer.transform([i])
    similaridade = cosine_similarity(vector_comparacao, tfidf_matrix_universidades)
    similaridade_list.append(similaridade)

    max_value = max(similaridade_list)
    max_index = similaridade_list.index(max_value)

    print(float(max(max_value[0])))
    print(max_index)

    df2_edu['nome_corrigido'] = np.where(max(max_value[0]) >= threshold , 'aa', df['NOME-INSTITUICAO'])

In [ ]:


    # Substituir os nomes no df2 pelo nome correspondente do df1
    df2_edu['nome_corrigido'] = df2_edu.apply(
        lambda row: df_uni_filtered.iloc[row[max_index]]['NOME_DA_IES']
        if max(max_value[0]) >= threshold
        else df2_edu['NOME-INSTITUICAO'], axis=1
    )


In [ ]:
pd.set_option('display.max_rows', None)
print(df2_edu[['NOME-INSTITUICAO', 'nome_corrigido']].loc[df2_edu['NOME-INSTITUICAO'] != df2_edu['nome_corrigido']])
pd.set_option('display.max_rows', 10)

KeyError: "['nome_corrigido'] not in index"

In [ ]:
# SIMILARIDADE POR SIGLA
# NAO FUNCIONA, VER SE CONTEM

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Vetorização TF-IDF
vectorizer = TfidfVectorizer().fit(df_uni_filtered['SIGLA'].values)
tfidf_matrix_1 = vectorizer.transform(df_uni_filtered['SIGLA'].values)
tfidf_matrix_2 = vectorizer.transform(df2['NOME-INSTITUICAO'].values)

# Calcular a similaridade de cosseno
similaridades = cosine_similarity(tfidf_matrix_2, tfidf_matrix_1)

# Para cada universidade no df2, encontre o índice do nome mais similar no df1
df2['indice_mais_similar_sigla'] = similaridades.argmax(axis=1)

# Substituir os nomes no df2 pelo nome correspondente do df1
df2['nome_corrigido_sigla'] = df2['indice_mais_similar_sigla'].apply(lambda idx: df_uni_filtered.iloc[idx]['SIGLA'])

# Exibir o resultado final
print(df2[['NOME-INSTITUICAO', 'nome_corrigido_sigla']])

# SE TROCOU, ATRIBUIR A COLUNA 'ROTULO' COMO EDUCACAO
# SE NÃO, ATRIBUIR A COLUNA COMO OUTROS

### N-gram


In [ ]:
vectorizer_ngram = TfidfVectorizer(analyzer='char', ngram_range=(2, 4)).fit(df_uni_filtered['NOME_DA_IES'].values)
tfidf_matrix_1_ngram = vectorizer.transform(df_uni_filtered['NOME_DA_IES'].values)
tfidf_matrix_2_ngram = vectorizer.transform(df2['NOME-INSTITUICAO'].values)

similaridades_ngram = cosine_similarity(tfidf_matrix_2_ngram, tfidf_matrix_1_ngram)

# Para cada universidade no df2, encontre o índice do nome mais similar no df1
df2['indice_mais_similar_ngram'] = similaridades_ngram.argmax(axis=1)
df2['similaridade_maxima_ngram'] = similaridades_ngram.max(axis=1)

threshold = 0.8

# Substituir os nomes no df2 pelo nome correspondente do df1
df2['nome_corrigido_ngram'] = df2.apply(
    lambda row: df_uni_filtered.iloc[row['indice_mais_similar_ngram']]['NOME_DA_IES']
    if row['similaridade_maxima_ngram'] >= threshold
    else row['NOME-INSTITUICAO'], axis=1
)

# Exibir o resultado final
#df2[['NOME-INSTITUICAO', 'nome_corrigido']]
#df2.loc[df2['NOME-INSTITUICAO'] != df2['nome_corrigido']]

In [ ]:
pd.set_option('display.max_rows', 10)
print(df2[['NOME-INSTITUICAO', 'nome_corrigido_ngram']].loc[df2['NOME-INSTITUICAO'] != df2['nome_corrigido_ngram']])

### K-Means

In [ ]:
# CAIO

vectorizer = TfidfVectorizer().fit(df_uni_filtered['NOME_E_SIGLA'].values)
tfidf_matrix_1 = vectorizer.transform(df_uni_filtered['NOME_E_SIGLA'].values)

from sklearn.cluster import KMeans

# Step 4: Perform clustering (K-means as an example)
n_clusters = 2754  # Set your number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(tfidf_matrix_1)

# Step 5: Get cluster labels and assign each document to a cluster
cluster_labels = kmeans.labels_

for i in range(n_clusters):
    cluster_points = np.where(cluster_labels == i)[0]  # Get all points in cluster i
    cluster_center = kmeans.cluster_centers_[i]  # Centroid of cluster i

    # Find document closest to the centroid (most recurring)
    closest_doc_idx = np.argmax(cosine_similarity(tfidf_matrix_1[cluster_points], [cluster_center]))
    most_recurring_idx = cluster_points[closest_doc_idx]
    original_row_index = df2_edu.index[most_recurring_idx]
    most_recurring_document = df2_edu.loc[original_row_index]
    print(most_recurring_document['NOME-INSTITUICAO'])
most_recurring_document

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


UNIVERSIDADE FEDERAL CEARA
UNIVERSIDADE FEDERAL SAO CARLOS
UNIVERSIDADE FEDERAL AMAZONAS
UNIVERSIDADE FEDERAL RIO GRANDE
UNIVERSIDADE ESTADUAL CAMPINAS
FUNDACAO OSWALDO CRUZ
UNIVERSIDADE FEDERAL FLUMINENSE
UNIVERSIDADE PAULISTA
UNIVERSIDADE ESTADUAL PAULISTA JULIO MESQUITA FILHO
UNIVERSIDADE FEDERAL RIO JANEIRO
PONTIFICIA UNIVERSIDADE CATOLICA GOIAS
UNIVERSIDADE FEDERAL CEARA
UNIVERSIDADE ESTACIO SA
UNIVERSIDADE FEDERAL GOIAS
CONSELHO NACIONAL DESENVOLVIMENTO CIENTIFICO TECNOLOGICO
UNIVERSIDADE FEDERAL SANTA CATARINA
UNIVERSIDADE SAO PAULO
UNIVERSIDADE PAULISTA
PONTIFICIA UNIVERSIDADE CATOLICA MINAS GERAIS
UNIVERSIDADE FEDERAL RURAL PERNAMBUCO
UNIVERSIDADE FEDERAL SANTA MARIA
UNIVERSIDADE FEDERAL LAVRAS
UNIVERSIDADE ESTADO PARA
UNIVERSIDADE FEDERAL PARANA
UNIVERSIDADE FEDERAL ESPIRITO SANTO
UNIVERSIDADE ESTADO BAHIA
UNIVERSIDADE FEDERAL GOIAS
UNIVERSIDADE FEDERAL MATO GROSSO SUL
UNIVERSIDADE FEDERAL RIO GRANDE SUL
PONTIFICIA UNIVERSIDADE CATOLICA SAO PAULO
UNIVERSIDADE FEDERAL CAMPINA 

,17997
NRO-ID-CNPQ,3318505161162217
CODIGO-INSTITUICAO,008300000001
NOME-INSTITUICAO,UNIVERSIDADE FEDERAL PARAIBA
ANO-INICIO,2006
ANO-FIM,2024
ENQUADRAMENTO-FUNCIONAL,LIVRE
TIPO-DE-VINCULO,LIVRE
OUTRAS-INFORMACOES,ALUNA MATRICULADA NO CURSO DOUTORADO-PPGE/UFP
CARGA-HORARIA-SEMANAL,
FLAG-DEDICACAO-EXCLUSIVA,FALSE


### Word Embedding
- muito lentooo
- 50k linhas : 3min

In [ ]:
!python -m spacy download pt_core_news_sm
import spacy

# AJUSTAR O CODIGO PQ NAO TROCA A COLUNA NEM NADA

nlp = spacy.load("pt_core_news_sm")  # Carrega um modelo pré-treinado
df2['spacy_similarity'] = df2.apply(
    lambda row: nlp(row['NOME-INSTITUICAO']).similarity(nlp(df_uni_filtered['NOME_DA_IES'][row['indice_mais_similar']])),
    axis=1
)

df2

### Word2Vect

# CAIO

## MERGE SAME UNI

In [ ]:
# merge ANO-INICIO and ANO-FIM with same CODIGO-INSTITUICAO and NRO-ID-CNPQ

df_merged = df.groupby(['NRO-ID-CNPQ', 'CODIGO-INSTITUICAO']).agg(
    earliest_date1=('ANO-INICIO', 'min'),
    latest_date2=('ANO-FIM', 'max')
).reset_index()


# df.loc[df['NRO-ID-CNPQ'] == 3300778291054405]

In [ ]:
# merge column NOME-INSTITUICAO with df_merged

df_drop_duplicates = df.drop_duplicates(subset=['NRO-ID-CNPQ', 'CODIGO-INSTITUICAO'])
df_inst_merged = pd.merge(df_merged, df_drop_duplicates, on=['CODIGO-INSTITUICAO', 'NRO-ID-CNPQ'], how='left')
df_inst_merged['NOME-INSTITUICAO'] = df_inst_merged['NOME-INSTITUICAO'].str.upper()

df_inst_merged = df_inst_merged[['NRO-ID-CNPQ', 'CODIGO-INSTITUICAO', 'earliest_date1', 'latest_date2', 'NOME-INSTITUICAO']]
df_inst_merged

## NLP

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# The target word for filtering (single word)
target_word = "UNIVERSIDADE DE SÃO PAULO"

# Step 1: Turn phrases into TF-IDF vectors
vectorizer = TfidfVectorizer()
phrase_vectors = vectorizer.fit_transform(df_inst_merged['NOME-INSTITUICAO'])

# Step 2: Convert the target word into a vector
# We will use the same vectorizer to transform the target word
target_vector = vectorizer.transform([target_word])

# Step 3: Compute cosine similarity between the target word and all phrases
similarity_scores = cosine_similarity(target_vector, phrase_vectors).flatten()

df_inst_merged['PREDICTION'] = similarity_scores

# Step 4: Filter phrases based on a similarity threshold (e.g., > 0.1)
threshold = 0.7
filtered_phrases = df_inst_merged[similarity_scores > threshold]

# Display the phrases that have similarity above the threshold
#print("Filtered Phrases based on Cosine Similarity with the word 'sky':")
#print(filtered_phrases)
df_inst_merged

In [ ]:
df_inst_merged.to_csv("atuacoes_2_filtrado.csv", encoding='utf8')